<a href="https://colab.research.google.com/github/keylacampusano/Python-Practice/blob/main/LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install datasets
!pip install transformers
!pip install evaluate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.8 MB/s eta 0:00:00


In [5]:
!pip install rouge

In [6]:
import numpy as np
import nltk
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plot
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from datasets import load_dataset
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from transformers import pipeline
from sklearn.metrics import accuracy_score
from rouge import Rouge

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


# SENTIMENT ANALYSIS ::)

In [7]:
#load dataset from hf
sentiment_analysis_df = load_dataset("amazon_polarity")
sentiment_analysis_sample= sentiment_analysis_df["train"].select(range(10))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/3600000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/400000 [00:00<?, ? examples/s]

In [11]:
sentiment_analysis_samp = sentiment_analysis_df["train"].to_pandas()
sentiment_analysis_samp

,label,title,content
0,1,Stuning even for the non-gamer,This sound track was beautiful! It paints the ...
1,1,The best soundtrack ever to anything.,I'm reading a lot of reviews saying that this ...
2,1,Amazing!,This soundtrack is my favorite music of all ti...
3,1,Excellent Soundtrack,I truly like this soundtrack and I enjoy video...
4,1,"Remember, Pull Your Jaw Off The Floor After He...","If you've played the game, you know how divine..."
...,...,...,...
3599995,0,Don't do it!!,The high chair looks great when it first comes...
3599996,0,"Looks nice, low functionality",I have used this highchair for 2 kids now and ...
3599997,0,"compact, but hard to clean","We have a small house, and really wanted two o..."
3599998,0,what is it saying?,not sure what this book is supposed to be. It ...


In [20]:
#first train
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased-finetuned-sst-2-english")
model = AutoModelForSequenceClassification.from_pretrained("distilbert/distilbert-base-uncased-finetuned-sst-2-english")

In [21]:
sentiment_analysis_sampl = sentiment_analysis_df["train"].to_pandas()
sentiment_analysis_sampl.iloc[(range(10))]

,label,title,content
0,1,Stuning even for the non-gamer,This sound track was beautiful! It paints the ...
1,1,The best soundtrack ever to anything.,I'm reading a lot of reviews saying that this ...
2,1,Amazing!,This soundtrack is my favorite music of all ti...
3,1,Excellent Soundtrack,I truly like this soundtrack and I enjoy video...
4,1,"Remember, Pull Your Jaw Off The Floor After He...","If you've played the game, you know how divine..."
5,1,an absolute masterpiece,I am quite sure any of you actually taking the...
6,0,Buyer beware,"This is a self-published book, and if you want..."
7,1,Glorious story,I loved Whisper of the wicked saints. The stor...
8,1,A FIVE STAR BOOK,I just finished reading Whisper of the Wicked ...
9,1,Whispers of the Wicked Saints,This was a easy to read book that made me want...


In [22]:
import torch

analizer0 = pipeline(
    task="text-classification",
    model="distilbert/distilbert-base-uncased-finetuned-sst-2-english")
inputs = tokenizer(sentiment_analysis_sample["content"][6], return_tensors="pt")
with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]

'NEGATIVE'

In [29]:
analizer1 = pipeline(
    task= "text-classification",
    model = AutoModelForSequenceClassification.from_pretrained("SamLowe/roberta-base-go_emotions"),
    tokenizer = AutoTokenizer.from_pretrained("SamLowe/roberta-base-go_emotions"),
    min_length= 10,
    max_length=100,
)
inputs = tokenizer(sentiment_analysis_sample["content"][4], return_tensors="pt")
with torch.no_grad():
    logits = model(**inputs).logits
predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]


'POSITIVE'

In [32]:
analizer2= pipeline(
    task= "text-classification",
    model="finiteautomata/bertweet-base-sentiment-analysis",
    min_lenght= 20,
    max_lenght= 90,
    model_kwargs= {"cache_dir": "sentiment_analysis_model1/"}
  )
inputs = tokenizer(["me encanta tu vestido"], return_tensors="pt")
with torch.no_grad():
    logits = model(**inputs).logits
predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]



emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


'POSITIVE'

In [41]:
#know accuracy now
with torch.no_grad():
    logits = model(**inputs).logits
    predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]

test_texts = (sentiment_analysis_sampl.iloc[4])
test_labels = [sentiment_analysis_sampl.loc[7, "label"]]  # Etiquetas correspondientes a los textos de prueba

predicted_class_id = np.array([predicted_class_id])

accuracy = accuracy_score(test_labels, predicted_class_id)
print("Accuracy:", accuracy)


Accuracy: 1.0


# SUMMARIZATION

In [ ]:
#TAKE DATASET FRUM HF

summarization_df = load_dataset("samsum")
summarization_sample = summarization_df["train"].select(range(10))

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

In [ ]:
display(summarization_sample.to_pandas())

,id,dialogue,summary
0,13818513,Amanda: I baked cookies. Do you want some?\r\...,Amanda baked cookies and will bring Jerry some...
1,13728867,Olivia: Who are you voting for in this electio...,Olivia and Olivier are voting for liberals in ...
2,13681000,"Tim: Hi, what's up?\r\nKim: Bad mood tbh, I wa...",Kim may try the pomodoro technique recommended...
3,13730747,"Edward: Rachel, I think I'm in ove with Bella....",Edward thinks he is in love with Bella. Rachel...
4,13728094,Sam: hey overheard rick say something\r\nSam:...,"Sam is confused, because he overheard Rick com..."
5,13716343,"Neville: Hi there, does anyone remember what d...",Wyatt reminds Neville his wedding anniversary ...
6,13611672,John: Ave. Was there any homework for tomorrow...,John didn't show up for class due to some work...
7,13730463,Sarah: I found a song on youtube and I think y...,Sarah sends James an instrumental song he migh...
8,13809976,Noah: When and where are we meeting? :)\r\nMad...,"Noah wants to meet, he quit his job, because h..."
9,13809912,Matt: Do you want to go for date?\r\nAgnes: Wo...,Matt invites Agnes for a date to get to know e...


In [ ]:
#Primer train con modelo de HF
summarizer0 = pipeline(
    task= "summarization",
    model= "cnicu/t5-small-booksum",
)

In [ ]:
summarization_sample["dialogue"][3]

"Edward: Rachel, I think I'm in ove with Bella..\r\nrachel: Dont say anything else..\r\nEdward: What do you mean??\r\nrachel: Open your fu**ing door.. I'm outside"

In [ ]:
summarization_sample["summary"][3]

'Edward thinks he is in love with Bella. Rachel wants Edward to open his door. Rachel is outside. '

In [ ]:
summarizer0(summarization_sample["dialogue"][3])

Your max_length is set to 200, but your input_length is only 57. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=28)


[{'summary_text': "Rachel: Rachel, I think I'm in ove with Bella.. Edward: What do you mean?? rachel: Open your fu**ing door.."}]

In [ ]:
#Segundo train con modelo de HF
summarizer1 = pipeline(
    task="summarization",
    model="sshleifer/distilbart-cnn-12-6",

)


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [ ]:
summarization_sample["dialogue"][4]

"Sam: hey  overheard rick say something\r\nSam: i don't know what to do :-/\r\nNaomi: what did he say??\r\nSam: he was talking on the phone with someone\r\nSam: i don't know who\r\nSam: and he was telling them that he wasn't very happy here\r\nNaomi: damn!!!\r\nSam: he was saying he doesn't like being my roommate\r\nNaomi: wow, how do you feel about it?\r\nSam: i thought i was a good rommate\r\nSam: and that we have a nice place\r\nNaomi: that's true man!!!\r\nNaomi: i used to love living with you before i moved in with me boyfriend\r\nNaomi: i don't know why he's saying that\r\nSam: what should i do???\r\nNaomi: honestly if it's bothering you that much you should talk to him\r\nNaomi: see what's going on\r\nSam: i don't want to get in any kind of confrontation though\r\nSam: maybe i'll just let it go\r\nSam: and see how it goes in the future\r\nNaomi: it's your choice sam\r\nNaomi: if i were you i would just talk to him and clear the air"

In [ ]:
summarizer1(summarization_sample["dialogue"][4])

[{'summary_text': ' Sam overheard rick say something: "He doesn\'t like being my roommate" Sam: "I don\'t know what to do :-/ He was talking on the phone with someone," Sam says . Sam: i used to love living with you before i moved in with me boyfriend .'}]

In [ ]:
#USE THE METRICS FOR THIS TASK (ROUGE (Recall-Oriented Understudy for Gisting Evaluation))

reference= summarization_sample["dialogue"][1]
hypothesis= summarization_sample["summary"][1]

rouge = Rouge()

scores= rouge.get_scores(hypothesis, reference)

print(scores)

[{'rouge-1': {'r': 0.3125, 'p': 0.45454545454545453, 'f': 0.3703703655418381}, 'rouge-2': {'r': 0.11764705882352941, 'p': 0.2, 'f': 0.14814814348422511}, 'rouge-l': {'r': 0.3125, 'p': 0.45454545454545453, 'f': 0.3703703655418381}}]


**TRANSLATION**

In [8]:
#TRANSLATION
translation_df = load_dataset("bsd_ja_en")
translation_sample = translation_df["train"]



Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2120 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2051 [00:00<?, ? examples/s]

In [9]:
display(translation_sample.to_pandas())

,id,tag,title,original_language,no,en_speaker,ja_speaker,en_sentence,ja_sentence
0,190329_J07_03,phone call,伝言への折り返し電話 打ち合わせ日程調整,ja,1,Doi-san,土井さん,Hi this is the systems development department ...,はい、K社システム開発部です。
1,190329_J07_03,phone call,伝言への折り返し電話 打ち合わせ日程調整,ja,2,Takaichi-san,高市さん,My name is Takaichi from Company H.,H社の高市と申します。
2,190329_J07_03,phone call,伝言への折り返し電話 打ち合わせ日程調整,ja,3,Takaichi-san,高市さん,Thank you as always.,いつもお世話になっております。
3,190329_J07_03,phone call,伝言への折り返し電話 打ち合わせ日程調整,ja,4,Doi-san,土井さん,Thank you as always as well.,こちらこそ、お世話になっております。
4,190329_J07_03,phone call,伝言への折り返し電話 打ち合わせ日程調整,ja,5,Takaichi-san,高市さん,Is Inada-san there?,稲田さんはいらっしゃいますか？
...,...,...,...,...,...,...,...,...,...
19995,190315_E001_01,meeting,Meeting: Wrap-up meeting (after meeting a star...,en,28,Mr. Bob Cook,ボブ クックさん,I'll finish it by 5 o'clock today.,今日の5時までに書き上げます。
19996,190315_E001_01,meeting,Meeting: Wrap-up meeting (after meeting a star...,en,29,Mr. Al Johnson,アル ジョンソンさん,Perfect.,完璧だね。
19997,190315_E001_01,meeting,Meeting: Wrap-up meeting (after meeting a star...,en,30,Mr. Al Johnson,アル ジョンソンさん,I'll go talk to the general manager tomorrow t...,明日ゼネラルマネージャーにA社の件を話しておきますね。
19998,190315_E001_01,meeting,Meeting: Wrap-up meeting (after meeting a star...,en,31,Mr. Al Johnson,アル ジョンソンさん,Hopefully he will like Company A and make the ...,彼がA社を気に入ってくれて、投資できる方向に決まることを願いましょう。


In [11]:
#PRIMER TRAIN CON LA TERCERA TAREA (TRANSLATION)
from nltk.translate.bleu_score import sentence_bleu
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


In [9]:
translator0 = pipeline(
    task="translation",
    tokenizer=AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-pl-en"),
    model=AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-pl-en"),
    min_length=20,
    max_length= 100,
    model_kwargs={"cache_dir": "translator_model1/"}

)

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [13]:
#Pongamoslo a prueba

translator0(translation_sample["en_sentence"][0])

[{'translation_text': "♪ Hi this is the systems development department of Company K. ♪ ♪ I don't know what you're talking about ♪"}]

In [14]:
#Pongamoslo a prueba

translator0(translation_sample["ja_sentence"][0])

[{'translation_text': '♪ ♪ ♪ Ooh, ooh, ooh, ooh, ooh ♪ ♪ Ooh, ooh, ooh, ooh, ooh ♪ ♪ Ooh, ooh, ooh, ooh, ooh, ooh, ooh, ooh, ooh, ooh, ooh, oo'}]

In [21]:
#METRIC BLEU
#involves using a machine translation system to generate translations and then calculating the BLEU score to evaluate the quality of those translations.

reference = ["is your name keyla?"]
hypothesis = ["tu nombre es keyla?"]

reference = [nltk.word_tokenize(sent) for sent in reference]
hypothesis = nltk.word_tokenize(hypothesis[0])

bleu_score = sentence_bleu(reference, hypothesis)


print("BLEU Score:", bleu_score)

BLEU Score: 8.38826642100846e-155


In [6]:
#SEGUNDO TRAINING CON OTRO MODEL :)

translator1= pipeline(
    task= "translation",
    model="google-t5/t5-small",
    model_kwargs={"cache_dir": "translator_model2/"}
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/__init__.py:1086: UserWarning: "translation" task was used, instead of "translation_XX_to_YY", defaulting to "translation_en_to_de"
  warnings.warn(


In [14]:
#otra pruebita
reference = translation_sample["en_sentence"][0]
hypothesis = translation_sample["ja_sentence"][1]

reference = [nltk.word_tokenize(sent) for sent in reference]
hypothesis = nltk.word_tokenize(hypothesis[0])

bleu_score = sentence_bleu(reference, hypothesis)


print("BLEU Score:", bleu_score)

BLEU Score: 1.821831989445342e-231


In [15]:
translator1(translation_sample["en_sentence"][0])

[{'translation_text': 'Hallo, das ist die Abteilung für Systementwicklung der Firma K.'}]

In [17]:
translator1(translation_sample["ja_sentence"][1])

[{'translation_text': 'H'}]